In [124]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer", "viridis")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained
source("~/oceanography/scripts/format_genetic_parentage_matrix.R")
source("~/oceanography/scripts/format_genetic_kernel_parentage_matrix.R")
source("~/oceanography/scripts/format_biophys_normalized_matrix.R")
source("~/oceanography/scripts/format_biophys_parentage_matrix.R")

#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the necessary input data for likelihood functions
load(file= "~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_InputSurveyData.Rdata")
load(file= "~/oceanography/script_output/ROMSDataTables/2021-11-04_SeededParticleInfo.Rdata")
load(file="~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_AddDestTables.Rdata")

In [125]:
TotalPar2012_4 <- fread(file="~/oceanography/empirical_data/genetics/parentage_table_2012-14.csv")
TotalPar2012_4 <- unique(TotalPar2012_4[offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
            , n_matches := sum(n_matches), by=c("offs_site", "par_site", "year")], by=c("offs_site", "par_site", "year"))

MonsoonRecSampPar <- unique(fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")[
    , .(offs_site=site, par_site, monsoon=season)][offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
    , num_gen := (.N), by=c("offs_site", "monsoon")][
    !is.na(par_site), n_matches := as.numeric((.N)), by=c("offs_site", "par_site", "monsoon")], by=c("offs_site", "par_site", "monsoon"))[is.na(n_matches), n_matches:=0]

genetic_parentage_matrices <- format_genetic_parentage_matrix(TotalPar2012_4, MonsoonRecSampPar, SurveyData)
#fwrite(genetic_parentage_matrices$GenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2012, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2013, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2014, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixSWM2012-14ForROMSComp.csv")


genetic_kernel_matrices <- format_genetic_kernel_parentage_matrix(TotalPar2012_4, MonsoonRecSampPar, SurveyData)

#fwrite(genetic_kernel_matrices$KernelGenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2012, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2013, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2014, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv")


__read in the biophysical data and format for likelihood functions__

In [122]:
SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableCompleteMetaLongForm08DayPLD.csv")[year %in% c(2012, 2013, 2014) & dest != "CAI"] #filter out CAI as a destination for now, not very well spatially defined
total_release_days

[1] 687

In [16]:

biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConn, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestAnnualSim=AddDestAnnualSim, AddDestMonsoonSim=AddDestMonsoonSim)

#fwrite(biophys_norm_matrices$FullBiophysMatNorm, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2013ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2014ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormNEM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixNEM2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormSWM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixSWM2012-14ForROMSComp15DayPLD.csv")

biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConn, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestGenAnnual=AddDestGenAnnual)
#fwrite(biophys_parentage_matrices$FullBiophysMat, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2013ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2014ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatNEM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatSWM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2012-14ForROMSComp15DayPLD.csv")

In [129]:
#get the best-fit kernels and profile likelihood confidence intervals in fit_data.ipynb


In [126]:
AllFishObsWithPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")[year %in% c("2012", "2013", "2014")][offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"] #collapse Magbangon values

#bootstrap the particle AND parentage data and re-fit kernels and perform bode likelihood function for dispersal event matching
num_samples_sim <- nrow(SimConn) #because each row in SimConn is a particle the number of rows is N samples per iteration for bootstrapping
#num_samples_par <- sum(genetic_kernel_matrices$KernelGenMat2012_4[1:nrow(genetic_kernel_matrices$KernelGenMat2012_4)-1,]) #N samples per iteration for bootstrapping the genetic parentage data is the total number of recruits samp
num_samples_unassigned <- sum(genetic_kernel_matrices$KernelGenMat2012_4[nrow(genetic_kernel_matrices$KernelGenMat2012_4),]) #N samples per iteration for bootstrapping the genetic parentage data is the total number of recruits sampled
num_samples_monsoon_genotyped <- sum(unique(MonsoonRecSampPar, by=c("offs_site", "monsoon"))[, num_gen])
num_samples_monsoon_par <- sum(unique(MonsoonRecSampPar, by=c("offs_site","par_site", "monsoon"))[, n_matches])

#make table to hold results
BootKernels <- data.table(data_source=character(), time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), log_like=numeric(), iteration=numeric())

#pb <- txtProgressBar(min = 0, max = 1000, style = 3)

#for(i in 1:1000){
    
    #resample dispersal data, both genetics and simulation
    SimConnInt2 <- SimConn[, .SD[sample(.N, num_samples_sim, replace=TRUE)]] #sample the particle data
    
#make parentage tables long form to sample rows for bootstrapping
    TotalParLongFormInt2 <- AllFishObsWithPar[matched_offs =="Y", num_matches_annual := .N, by=c("year", "offs_site", "par_site")][
        is.na(num_matches_annual), num_matches_annual := 0][
        year %in% c("2012", "2013", "2014"), .SD[sample(.N, num_matches_annual, replace=TRUE)], by=c("year", "offs_site", "par_site")][
        ,-"num_matches_annual"]#num_matches_annual is used as the number of rows to draw for each source/dest/year combo, then make a summary table and drop that column now that the data is sampled 
    #nrow(TotalParLongFormInt2[matched_offs=="Y"]) #should be 37
    TotalPar2012_4Int3 <- TotalParLongFormInt2[, .(n_matches=.N), by=c("offs_site", "par_site", "year")] 
    
    AllRecruitsSampledLongForm <- AllFishObsWithPar[is.na(par_site)][, num_unassigned_annual := .N, by=c("year", "site")][
        is.na(num_unassigned_annual), num_unassigned_annual := 0][
        , .SD[sample(.N, num_unassigned_annual, replace=TRUE)], by=c("year", "site")][
        ,-c("num_unassigned_annual", "num_matches_annual")]#num_sampled_annual is used as the number of rows to draw for each dest/year combo, then make a summary table and drop that column now that the data is sampled 
    #nrow(AllRecruitsSampledLongForm)#should be 357
    
    AllRecruitsSampledInt <- rbind(TotalParLongFormInt2, AllRecruitsSampledLongForm)[,.(n_offs_gen=.N), by=c("year", "site")] #use the values in this table to re-make SurveyData
    #sum(AllRecruitsSampledInt$n_offs_gen)#should be 394
    SurveyDataInt <- left_join(SurveyData[year %in% c("2012", "2013", "2014"),-"n_offs_gen"], AllRecruitsSampledInt, by=c("year", "site"))[is.na(n_offs_gen), n_offs_gen := 0] 
    #sum(SurveyDataInt$n_offs_gen) #should be 394
    
    MonsoonRecSampParLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$n_matches), 1:ncol(MonsoonRecSampPar)][, -"n_matches"]
    #nrow(SurveyDataLongForm) #should be 357
    #nrow(TotalParLongForm)#should be 37
    #nrow(MonsoonRecSampParLongForm) #should be 17   
    
    #TotalParLongFormInt2 <- TotalParLongForm[, .SD[sample(.N, num_samples_par, replace=TRUE)]]
    
    #for monsoon genotyped recruits
    MonsoonRecSampLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$num_gen), 1:ncol(MonsoonRecSampPar)][, -c("num_gen", "n_matches")]
    MonsoonRecSampLongFormInt2 <- MonsoonRecSampLongForm[, .SD[sample(.N, num_samples_monsoon_genotyped, replace=TRUE)]]
    
    #for monsoon assigned
    MonsoonRecSampParLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$n_matches), 1:ncol(MonsoonRecSampPar)][, -c("num_gen", "n_matches")]   
    MonsoonRecSampParLongFormInt2 <- MonsoonRecSampParLongForm[, .SD[sample(.N, num_samples_monsoon_par, replace=TRUE)]]
    
    #check
    #nrow(MonsoonRecSampParLongFormInt)+nrow(MonsoonRecSampLongFormInt)#should be 273 because the parentage assignments are included in the num_genotyped for this input table
    
    #turn back into summary table
    MonsoonRecSampParInt3 <- left_join(MonsoonRecSampLongFormInt2[, .(num_gen=.N), by=c("offs_site",  "monsoon")], MonsoonRecSampParLongFormInt2[, .(n_matches=.N), by=c("offs_site", "par_site", "monsoon")], by=c("offs_site", "monsoon"))[, n_matches := as.numeric(n_matches)]    
    
    #sum(MonsoonRecSampParInt3$n_matches, na.rm = T)#should be 17
    #sum(unique(MonsoonRecSampParInt3,by=c("offs_site", "monsoon"))[, num_gen]) #should be 256
    #nrow(TotalParLongFormInt3)#should be 37
    #check that the resampling happened, and the before and after don't match
    #TotalParLongForm==TotalParLongFormInt3

    
    #TotalPar2012_4Int3 <- TotalParLongFormInt2[, .(n_matches=.N), by=c("offs_site", "par_site", "year")]
    #check totals 
    #sum(TotalPar2012_4Int3$n_matches)
    
    #the unassigned row comes from SurveyData table. may need to revisit this section after testing- should AddDest tables be remade every iteration, within the function?
    #SurveyDataLongForm <- SurveyData[rep(seq_len(nrow(SurveyData)), SurveyData$n_offs_gen), 1:ncol(SurveyData)][, -"n_offs_gen"]
    #nrow(SurveyDataLongForm[year %in% c("2012", "2013", "2014")])#should be 394

    #turn back into summary table
    #SurveyDataInt <- SurveyDataLongForm[, .(n_offs_gen=.N), by=c("year", "site", "num_females", "prop_anem_samp")]
        
        #format the matrices for kernel fitting and likelihood function
        genetic_parentage_matrices <- format_genetic_parentage_matrix(TotalPar2012_4=TotalPar2012_4Int3, MonsoonRecSampPar=MonsoonRecSampParInt3, SurveyData=SurveyDataInt)
        genetic_kernel_matrices <- format_genetic_kernel_parentage_matrix(TotalPar2012_4=TotalPar2012_4Int3, MonsoonRecSampPar=MonsoonRecSampParInt3, SurveyData=SurveyDataInt)
        biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestAnnualSim=AddDestAnnualSim, AddDestMonsoonSim=AddDestMonsoonSim)
        biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestGenAnnual=AddDestGenAnnual)


[1] 37

[1] 181

Cabatoan,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Lonas,Visca,Wangag
0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0
4,10,13,9,7,0,1,1,15


In [27]:
#I think to iterate through the data I should use the same table I made for the null model in the connectivity variation paper
AllFishObsWithPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")

head(AllFishObsWithPar)
head(TotalPar2012_4)
head(SurveyData)

fish_indiv,size,color,sex,gen_id,sample_id,site,date,anem_obs_time,anem_obs,gps,dive_type,year,input,par_fish_indiv,offs_site,par_site,year_match,matched_offs
762,5.1,Y,J,762,APCL12_256,Wangag,2012-05-14,09:45:00,NA,1,E,2012,offspring,236,Wangag,Palanas,2012,Y
2318,4.1,Y,J,2318,APCL12_275,Wangag,2012-05-14,11:35:00,NA,1,E,2012,offspring,NA,NA,NA,NA,N
409,4.5,Y,J,409,APCL12_276,Wangag,2012-05-14,11:44:00,NA,1,E,2012,offspring,NA,NA,NA,NA,N
2317,5.1,W,J,2317,APCL12_168,S.Magbangon,2012-05-10,14:08:00,NA,1,E,2012,offspring,NA,NA,NA,NA,N
2072,4.1,NA,NA,2072,APCL12_090,Palanas,2012-05-09,13:01:00,NA,1,E,2012,offspring,NA,NA,NA,NA,N
92,5.4,NA,NA,92,APCL12_094,Palanas,2012-05-09,12:46:00,NA,1,E,2012,offspring,NA,NA,NA,NA,N


offs_site,par_site,year,n_matches
Cabatoan,Magbangon,2014,1
Elementary School,Palanas,2013,2
Haina,Magbangon,2014,1
Magbangon,Magbangon,2013,1
Magbangon,Magbangon,2014,1
Magbangon,Palanas,2013,2


year,site,num_females,n_offs_gen,prop_anem_samp
2012,Cabatoan,8,4,0.4230769
2012,Caridad Cemetery,NA,0,0.0000000
2012,Caridad Proper,NA,0,0.0000000
2012,Elementary School,NA,0,0.0000000
2012,Gabas,NA,0,0.0000000
2012,Haina,NA,0,0.0000000


In [28]:
#so I think what I need to do is resample the AllFishObs table for the same number of genotyped recruits and parentage matches by year (2 separate tables) and then transform into the TotalPar input table and the survey data input

In [104]:


TotalParLongFormInt2 <- AllFishObsWithPar[matched_offs =="Y", num_matches_annual := .N, by=c("year", "offs_site", "par_site")][
    is.na(num_matches_annual), num_matches_annual := 0][
    year %in% c("2012", "2013", "2014"), .SD[sample(.N, num_matches_annual, replace=TRUE)], by=c("year", "offs_site", "par_site")][
    ,-"num_matches_annual"]#num_matches_annual is used as the number of rows to draw for each source/dest/year combo, then make a summary table and drop that column now that the data is sampled 
nrow(TotalParLongFormInt2[matched_offs=="Y"]) #should be 37
#TotalPar2012_4Int3 <- TotalParLongFormInt2[, .(n_matches=.N), by=c("offs_site", "par_site", "year")] 

AllRecruitsSampledLongForm <- AllFishObsWithPar[is.na(par_site)][, num_unassigned_annual := .N, by=c("year", "site")][
    is.na(num_unassigned_annual), num_unassigned_annual := 0][
    , .SD[sample(.N, num_unassigned_annual, replace=TRUE)], by=c("year", "site")][
    ,-c("num_unassigned_annual", "num_matches_annual")]#num_sampled_annual is used as the number of rows to draw for each dest/year combo, then make a summary table and drop that column now that the data is sampled 
#nrow(AllRecruitsSampledLongForm)#should be 357

AllRecruitsSampledInt <- rbind(TotalParLongFormInt2, AllRecruitsSampledLongForm)[,.(n_offs_gen=.N), by=c("year", "site")] #use the values in this table to re-make SurveyData
#sum(AllRecruitsSampledInt$n_offs_gen)#should be 394
SurveyDataInt <- left_join(SurveyData[year %in% c("2012", "2013", "2014"),-"n_offs_gen"], AllRecruitsSampledInt, by=c("year", "site"))[is.na(n_offs_gen), n_offs_gen := 0] 
#sum(SurveyDataInt$n_offs_gen) #should be 394

[1] 37

[1] 394

[1] 394

[1] 394

In [67]:
nrow(AllRecruitsSampled[matched_offs=="N"])

[1] 307

In [168]:
        genetic_parentage_matrices <- format_genetic_parentage_matrix(TotalPar2012_4=TotalPar2012_4Int3, MonsoonRecSampPar=MonsoonRecSampParInt3, SurveyData=SurveyDataInt)


ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 has 8 columns, inconsistent with item 1 which has 15 columns. To fill missing columns use fill=TRUE.


In [159]:
sum(SurveyDataInt$n_offs_gen)

[1] 791

In [176]:
TotalPar2012_4 <- TotalPar2012_4Int3
MonsoonRecSampPar <- MonsoonRecSampParInt3
SurveyData <- SurveyDataInt
TotalParInt <- unique(TotalPar2012_4[, `:=`(num_matches, 
        sum(n_matches)), by = c("year", "offs_site")][, -"par_site"], 
        by = c("year", "offs_site"))[, -"n_matches"]
    TotalUnassigned2012_4 <- TotalParInt[SurveyData[year %in% 
        c(2012, 2013, 2014)], on = .(offs_site = site, year)]
    TotalUnassigned2012_4[is.na(TotalUnassigned2012_4)] <- 0
    TotalUnassigned2012_4 <- TotalUnassigned2012_4[, `:=`(num_unassigned, 
        n_offs_gen - num_matches), by = c("year", "offs_site")][, 
        .(offs_site, year, num_unassigned)]
   # AddDestGen <- rbindlist(list(unique(cbind(SurveyData[year %in% 
   #     c(2012, 2013, 2014)][, .(offs_site = site)], SurveyData[year %in% 
   #     c(2012, 2013, 2014)][, .(par_site = site)]), by = c("offs_site", 
   #     "par_site"))[, `:=`(year, 2012)], unique(cbind(SurveyData[year %in% 
   #     c(2012, 2013, 2014)][, .(offs_site = site)], SurveyData[year %in% 
   #     c(2012, 2013, 2014)][, .(par_site = site)]), by = c("offs_site", 
   #     "par_site"))[, `:=`(year, 2013)], unique(cbind(SurveyData[year %in% 
   #     c(2012, 2013, 2014)][, .(offs_site = site)], SurveyData[year %in% 
   #     c(2012, 2013, 2014)][, .(par_site = site)]), by = c("offs_site", 
   #     "par_site"))[, `:=`(year, 2014)]))
    TotalPar2012_4 <- rbind(AddDestGen[!TotalPar2012_4, on = .(par_site, 
        offs_site, year)][, `:=`(n_matches, 0)], TotalPar2012_4[, 
        -"num_matches"])
    GenMat2012 <- as.matrix(rbind(dcast(TotalPar2012_4[year == 
        2012, .(par_site, offs_site, n_matches)][order(par_site, 
        offs_site)], par_site ~ offs_site, value.var = "n_matches")[, 
        -"par_site"], t(as.matrix(TotalUnassigned2012_4[year == 
        2012][order(offs_site)][, .(num_unassigned)])), use.names = FALSE))
    GenMat2012[is.na(GenMat2012)] <- 0
    GenMat2013 <- as.matrix(rbind(dcast(TotalPar2012_4[year == 
        2013, .(par_site, offs_site, n_matches)][order(par_site, 
        offs_site)], par_site ~ offs_site, value.var = "n_matches")[, 
        -"par_site"], t(as.matrix(TotalUnassigned2012_4[year == 
        2013][order(offs_site)][, .(num_unassigned)])), use.names = FALSE))
    GenMat2013[is.na(GenMat2013)] <- 0
    GenMat2014 <- as.matrix(rbind(dcast(TotalPar2012_4[year == 
        2014, .(par_site, offs_site, n_matches)][order(par_site, 
        offs_site)], par_site ~ offs_site, value.var = "n_matches")[, 
        -"par_site"], t(as.matrix(TotalUnassigned2012_4[year == 
        2014][order(offs_site)][, .(num_unassigned)])), use.names = FALSE))
    GenMat2014[is.na(GenMat2014)] <- 0
    GenMat2012_4 <- as.matrix(rbind(dcast(TotalPar2012_4[, .(par_site, 
        offs_site, n_matches)][order(par_site, offs_site)], par_site ~ 
        offs_site, value.var = "n_matches", fun.aggregate = sum)[, 
        -"par_site"], t(as.matrix(TotalUnassigned2012_4[, .(num_unassigned = sum(num_unassigned)), 
        by = "offs_site"][order(offs_site)][, .(num_unassigned)])), 
        use.names = FALSE))
    GenMat2012_4[is.na(GenMat2012_4)] <- 0
    MonsoonRecSampPar <- unique(MonsoonRecSampPar, by = c("offs_site", 
        "par_site", "monsoon"))[is.na(par_site), `:=`(n_matches, 
        0), by = c("offs_site", "par_site", "monsoon")][, .(offs_site, 
        par_site, monsoon, num_gen, n_matches)]
    MonsoonRecSamp <- unique(MonsoonRecSampPar, by = c("offs_site", 
        "monsoon"))[, -c("par_site", "n_matches")]
    ParMonsoon <- MonsoonRecSampPar[n_matches > 0, -"num_gen"]
    MonsoonParInt <- ParMonsoon[, .(num_matches = sum(n_matches)), 
        by = c("monsoon", "offs_site")]
    MonsoonUnassigned <- MonsoonParInt[MonsoonRecSamp, on = .(offs_site, 
        monsoon)]
    MonsoonUnassigned[is.na(MonsoonUnassigned)] <- 0
    MonsoonUnassigned[, `:=`(num_unassigned, num_gen - num_matches)]
    MonsoonUnassigned <- MonsoonUnassigned[, .(offs_site, monsoon, 
        num_unassigned)]
    #AddDestGen <- unique(cbind(SurveyData[year %in% c(2012, 2013, 
    #    2014)][, .(offs_site = site)], SurveyData[year %in% c(2012, 
    #    2013, 2014)][, .(par_site = site)]), by = c("offs_site", 
    #    "par_site"))
    AddDestGenMonsoon <- rbind(cbind(AddDestGen, data.table(monsoon = rep("NEM", 
        nrow(AddDestGen)))), cbind(AddDestGen, data.table(monsoon = rep("SWM", 
        nrow(AddDestGen)))))
    ParMonsoon <- rbind(AddDestGenMonsoon[!ParMonsoon, on = .(par_site, 
        offs_site, monsoon)][, `:=`(n_matches, 0)], ParMonsoon)
    MonsoonUnassigned <- rbind(AddDestGenMonsoon[!MonsoonUnassigned, 
        on = .(offs_site, monsoon)][, `:=`(num_unassigned, 0)][, 
        -"par_site"], MonsoonUnassigned)
    GenMatNEM <- rbind(dcast(ParMonsoon[monsoon == "NEM"][, .(par_site, 
        offs_site, n_matches)][order(par_site, offs_site)], par_site ~ 
        offs_site, value.var = "n_matches")[, -"par_site"], t(as.matrix(MonsoonUnassigned[monsoon == 
        "NEM"][order(offs_site)][, .(num_unassigned)])), use.names = FALSE)
    GenMatNEM[is.na(GenMatNEM)] <- 0
    GenMatSWM <- rbind(dcast(ParMonsoon[monsoon == "SWM"][, .(par_site, 
        offs_site, n_matches)][order(par_site, offs_site)], par_site ~ 
        offs_site, value.var = "n_matches")[, -"par_site"], t(as.matrix(MonsoonUnassigned[monsoon == 
        "SWM"][order(offs_site)][, .(num_unassigned)])), use.names = FALSE)
    GenMatSWM[is.na(GenMatSWM)] <- 0
    matrices <- list(GenMat2012 = GenMat2012, GenMat2013 = GenMat2013, 
        GenMat2014 = GenMat2014, GenMatNEM = GenMatNEM, GenMatSWM = GenMatSWM, 
        GenMat2012_4 = GenMat2012_4)

ERROR: Error in `[.data.table`(AddDestGen, !TotalPar2012_4, on = .(par_site, : Column(s) [year] not found in x


In [170]:
as.matrix(rbind(dcast(TotalPar2012_4[year == 
        2012, .(par_site, offs_site, n_matches)][order(par_site, 
        offs_site)], par_site ~ offs_site, value.var = "n_matches")[, 
        -"par_site"], t(as.matrix(TotalUnassigned2012_4[year == 
        2012][order(offs_site)][, .(num_unassigned)])), use.names = FALSE))

ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 has 8 columns, inconsistent with item 1 which has 15 columns. To fill missing columns use fill=TRUE.


In [174]:
dim(dcast(TotalPar2012_4[year == 
        2012, .(par_site, offs_site, n_matches)][order(par_site, 
        offs_site)], par_site ~ offs_site, value.var = "n_matches"))
TotalUnassigned2012_4

[1] 15 16

offs_site,year,num_unassigned
Cabatoan,2012,4
Magbangon,2012,10
Palanas,2012,13
Poroc Rose,2012,9
Poroc San Flower,2012,8
Sitio Lonas,2012,1
Visca,2012,1
Wangag,2012,14
Cabatoan,2013,3
Caridad Cemetery,2013,2


In [115]:
num_samples_monsoon_genotyped <- sum(unique(MonsoonRecSampPar, by=c("offs_site", "monsoon"))[, num_gen])
num_samples_monsoon_par <- sum(unique(MonsoonRecSampPar, by=c("offs_site","par_site", "monsoon"))[, n_matches])

#for genotyped recruits
MonsoonRecSampLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$num_gen), 1:ncol(MonsoonRecSampPar)][, -c("num_gen", "n_matches")]
MonsoonRecSampLongFormInt <- MonsoonRecSampLongForm[, .SD[sample(.N, num_samples_monsoon_genotyped, replace=TRUE)]]

#for assigned
MonsoonRecSampParLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$n_matches), 1:ncol(MonsoonRecSampPar)][, -c("num_gen", "n_matches")]   
MonsoonRecSampParLongFormInt <- MonsoonRecSampParLongForm[, .SD[sample(.N, num_samples_monsoon_par, replace=TRUE)]]

#check
#nrow(MonsoonRecSampParLongFormInt)+nrow(MonsoonRecSampLongFormInt)#should be 273 because the parentage assignments are included in the num_genotyped for this input table

#turn back into summary table
MonsoonRecSampParInt <- left_join(MonsoonRecSampLongFormInt[, .(num_gen=.N), by=c("offs_site",  "monsoon")], MonsoonRecSampParLongFormInt[, .(n_matches=.N), by=c("offs_site", "par_site", "monsoon")], by=c("offs_site", "monsoon"))   

#sum(MonsoonRecSampParInt$n_matches, na.rm = T)#should be 17
#sum(unique(MonsoonRecSampParInt,by=c("offs_site", "monsoon"))[, num_gen]) #should be 256

[1] 273

In [114]:
sum(MonsoonRecSampUnassignedLongFormInt[, .(num_gen=.N), by=c("offs_site",  "monsoon")][,num_gen])
MonsoonRecSampPar

[1] 239

offs_site,par_site,monsoon,num_gen,n_matches
Wangag,Palanas,SWM,58,2
Wangag,NA,SWM,58,0
Magbangon,NA,SWM,36,0
Palanas,NA,SWM,39,0
Poroc Rose,NA,SWM,5,0
Poroc San Flower,Wangag,SWM,5,1
Cabatoan,NA,SWM,8,0
Poroc San Flower,NA,SWM,5,0
Visca,NA,SWM,2,0
Poroc San Flower,Magbangon,SWM,5,1


In [124]:
MonsoonRecSampParInt <- left_join(MonsoonRecSampLongFormInt[, .(num_gen=.N), by=c("offs_site",  "monsoon")], MonsoonRecSampParLongFormInt[, .(n_matches=.N), by=c("offs_site", "par_site", "monsoon")], by=c("offs_site", "monsoon"))   

sum(MonsoonRecSampParInt$n_matches, na.rm = T)#should be 17
sum(unique(MonsoonRecSampParInt,by=c("offs_site", "monsoon"))[, num_gen]) #should be 256

[1] 17

[1] 256

In [36]:
sum(genetic_kernel_matrices$KernelGenMat2012_4[1:nrow(genetic_kernel_matrices$KernelGenMat2012_4)-1,])

[1] 37

In [22]:
TotalParInt
TotalUnassigned2012_4

offs_site,year,num_matches
Cabatoan,2014,1
Elementary School,2013,2
Haina,2014,1
Magbangon,2013,3
Magbangon,2014,2
Palanas,2013,10
Palanas,2014,1
Poroc San Flower,2012,2
Sitio Baybayon,2014,5
Visca,2013,1


offs_site,year,num_unassigned
Cabatoan,2012,4
Magbangon,2012,10
Palanas,2012,13
Poroc Rose,2012,9
Poroc San Flower,2012,7
San Agustin,2012,0
Sitio Lonas,2012,1
Visca,2012,1
Wangag,2012,15
Cabatoan,2013,3


In [ ]:
#bootstrap the particle data and re-fit kernels
num <- nrow(SimConn)

#make sure all possible routes are represented
AddDestSim <- rbind(rbindlist(list(unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2012],
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2013], 
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2014]))[, monsoon := "NEM"],
    rbindlist(list(unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2012],
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2013], 
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2014]))[, monsoon := "SWM"])

AddDestAllYearsSim <- unique(AddDestSim, by=c("source","dest"))[, -c("year","monsoon")] 

AddDestAnnualSim <- unique(AddDestSim, by=c("source","dest", "year"))[, -"monsoon"] 

AddDestMonsoonSim <- unique(AddDestSim[, -"year"], by=c("source", "dest", "monsoon"))
AddDestAnuualMonsoonSim <- unique(AddDestSim, by=c("source", "dest","year", "monsoon"))

##add in sites that aren't represented in every year by the genetics for the biophysical PARENTAGE matrix used to fit the biophysical kernel
AddDestGenAnnual <- rbindlist(list(unique(cbind(SurveyData[year==2012 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2012 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2012],  #what destinations were sampled, for use with unassigned table
unique(cbind(SurveyData[year==2013 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2013 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2013],
unique(cbind(SurveyData[year==2014 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2014 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2014]))

AddDestGen <- cbind(PropSamp[prop_anem_samp > 0 & year==2014, .(site)][, .(offs_site=site)], PropSamp[prop_anem_samp > 0 & year==2014, .(site)][, .(par_site=site)])

AddDestGenMonsoon <- rbind(cbind(AddDestGen, data.table(monsoon = rep("NEM",nrow(AddDestGen)))), cbind(AddDestGen, data.table(monsoon = rep("SWM",nrow(AddDestGen)))))

#read in all data needed for kernel fitting
#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv") 
Centroids$site <- gsub("_full", "", Centroids$site, fixed=TRUE)
Centroids$site <- gsub("_ten_per_cover", "", Centroids$site, fixed=TRUE)

Centroids <- Centroids %>%
    arrange(site)

#setorder(Centroids, site)#warning! This sets order based on site, and then lat/lon. So the table is not alphabetical by site, but that's fine as long as all of the "sampled_reef" vectors reflect this, so that reef_sizes, distance, and sampled_reefs match up by row/col index 
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- unique(fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")[
    , .(site, year=end_year, prop_anem_samp=total_prop_hab_sampled_anems_tidied)][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , prop_anem_samp := sum(prop_anem_samp), by=c("site", "year")], by=c("site", "year"))[ #collapse magbangons to match ROMS data
    site=="Sitio Lonas" & year %in% c(2012, 2013, 2014), prop_anem_samp :=1][site=="Caridad Proper" & year %in% c(2013, 2014), prop_anem_samp :=1]

### List of source locations
SitesSource <- Centroids

### List of destination locations
SitesDest <- Centroids

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- DistMatm*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
Area$site <- gsub("_ten_per_cover", "", Area$site, fixed=TRUE)

reef_sizes <- as.matrix(Area$kmsq)

#make a site index table, use this for Sampled_reefs input in kernel fitting
SiteIndex <- unique(Centroids %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel 
SiteIndexBioPhys <- unique(Centroids %>% arrange(site), by="site")[site != "CAI"][, index := .I] #add the row number as the unique site index, take CAI out for biophysical likelihood function

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0
#setnames(SurveyData, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#Allison's abundance time series data 
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "~/oceanography/empirical_data/genetics/females_df_F.RData")
load("~/oceanography/empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)
setnames(Abundance, "nF", "num_females")
Abundance <- unique(Abundance[site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
            , num_females := sum(num_females), by=c("site", "year")], by=c("site", "year"))
#join the survey sampling tables together
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0


SurveyData <- Abundance[, c("year", "site", "num_females")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this
#quick check that all components are in the same, alphabetical order
sum(which(SiteIndex$site==Area$site)==FALSE) #needs to be 0!! sites have to be in the same order
sum(which(Area$site==Centroids$site)==FALSE) #needs to be 0!! sites have to be in the same order

#start bootstrapping
SimKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), log_like=numeric(), iteration=numeric())

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

for(i in 1:1000){
    
    SimConnInt <- SimConn[, .SD[sample(.N, num, replace=TRUE)]] #sample the particle data
    
#format the matrices

    AllYearsRec <- SimConnInt[ , .(total_particles_rec = .N), by= c("source","dest")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
    AllYearsRelease <- unique(SimConnInt[, .(total_particles_released = as.numeric(daily_particles_released)*as.numeric(total_release_days)), by= c("source")], by="source") #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs
    
    AllYearsRecInt <- rbind(AddDestAllYearsSim[!AllYearsRec, on =.(source, dest)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
        , total_particles_rec:=0 ], AllYearsRec) 
    
    AllYearsReleaseInt <- rbind(AddDestAllYearsSim[!AllYearsRelease, on =.(source)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
        , total_particles_released:=0 ][, -"dest"], AllYearsRelease) 
    
    #collapse CAI and "Other" together into "unknown" before making the normalized matrices
    AllYearsRec <- unique(AllYearsRecInt[source== "Other" | source == "CAI", source := "unknown"][
        , total_particles_rec := sum(total_particles_rec),  by=c("source", "dest")], by=c("source", "dest"))
    AllYearsRelease <- unique(AllYearsReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
        , total_particles_released := sum(total_particles_released), by="source"], by="source")
    
    #join recruited and released tables together and make a column for the normalized values
    AllYearsNormConn <-  AllYearsRelease[AllYearsRec, on="source"][
        , source_norm_rec := total_particles_rec/total_particles_released]

    #make a parentage matrix for the whole biophysical results- CAI included with Other as "unknown", use this to fit a kernel
FullBiophysMat <- as.matrix(rbind(dcast(AllYearsNormConn[source %in% AddDestGenAnnual[, par_site] & dest %in% AddDestGenAnnual[, offs_site]][source !="unknown", .(source, dest, total_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="total_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
      dcast(AllYearsNormConn[dest %in% AddDestGenAnnual[, offs_site]][source  == "unknown", .(source, dest, total_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="total_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
FullBiophysMat[is.na(FullBiophysMat)] <- 0 #change NAs to zerosdim(FullBiophysMat)
#dim(FullBiophysMat)
#parentage matrix by years and monsoons
    
#make annual matrices with all of the particle data
AnnualRec <- SimConnInt[ , .(annual_particles_rec = .N), by= c("source","dest", "year")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
AnnualRelease <- unique(SimConnInt[, .(annual_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_annual)), by= c("source", "year")], by= c("source", "year")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

AnnualRecInt <- rbind(AddDestAnnualSim[!AnnualRec, on =.(source, dest, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , annual_particles_rec:=0 ], AnnualRec) 

AnnualReleaseInt <- rbind(AddDestAnnualSim[!AnnualRelease, on =.(source, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , annual_particles_released:=0 ][, -"dest"], AnnualRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
AnnualRec <- unique(AnnualRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , annual_particles_rec := sum(annual_particles_rec), by=c("source", "dest", "year")], by=c("source", "dest", "year"))
AnnualRelease <- unique(AnnualReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , annual_particles_released := sum(annual_particles_released), by=c("source", "year")], by=c("source", "year"))

#join recruited and released tables together and make a column for the normalized values
AnnualNormConn <-  AnnualRelease[AnnualRec, on=c("source", "year")][
    , source_norm_rec := annual_particles_rec/annual_particles_released]

AnnualBiophysMat2012 <- as.matrix(rbind(dcast(AnnualNormConn[year==2012][source %in% AddDestGenAnnual[year==2012, par_site] & dest %in% AddDestGenAnnual[year==2012, offs_site]][source !="unknown", .(source, dest, annual_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="annual_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(AnnualNormConn[year==2012][dest %in% AddDestGenAnnual[year==2012, offs_site]][source  == "unknown", .(source, dest, annual_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="annual_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
AnnualBiophysMat2012[is.na(AnnualBiophysMat2012)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2012)

AnnualBiophysMat2013 <- as.matrix(rbind(dcast(AnnualNormConn[year==2013][source %in% AddDestGenAnnual[year==2013, par_site] & dest %in% AddDestGenAnnual[year==2013, offs_site]][source !="unknown", .(source, dest, annual_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="annual_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(AnnualNormConn[year==2013][dest %in% AddDestGenAnnual[year==2013, offs_site]][source  == "unknown", .(source, dest, annual_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="annual_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
AnnualBiophysMat2013[is.na(AnnualBiophysMat2013)] <- 0 #change NAs to zeros
#dim(AnnualBiophysMat2013)

AnnualBiophysMat2014 <- as.matrix(rbind(dcast(AnnualNormConn[year==2014][source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]][source !="unknown", .(source, dest, annual_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="annual_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(AnnualNormConn[year==2014][dest %in% AddDestGenAnnual[year==2014, offs_site]][source  == "unknown", .(source, dest, annual_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="annual_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
AnnualBiophysMat2014[is.na(AnnualBiophysMat2014)] <- 0 #change NAs to zeros
#dim(AnnualBiophysMat2014)

MonsoonRec <- SimConnInt[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConnInt[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon")], by= c("source", "monsoon")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRec <- SimConnInt[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConnInt[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon")], by= c("source", "monsoon")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRecInt <- rbind(AddDestMonsoonSim[!MonsoonRec, on =.(source, dest, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_rec:=0 ], MonsoonRec) 

MonsoonReleaseInt <- rbind(AddDestMonsoonSim[!MonsoonRelease, on =.(source, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_released:=0 ][, -"dest"], MonsoonRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
MonsoonRec <- unique(MonsoonRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , monsoon_particles_rec := sum(monsoon_particles_rec), by=c("source", "dest", "monsoon")], by=c("source", "dest", "monsoon"))
MonsoonRelease <- unique(MonsoonReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , monsoon_particles_released := sum(monsoon_particles_released), by=c("source", "monsoon")], by=c("source", "monsoon"))

#join recruited and released tables together and make a column for the normalized values
MonsoonNormConn <-  MonsoonRelease[MonsoonRec, on=c("source", "monsoon")][
    , source_norm_rec := monsoon_particles_rec/monsoon_particles_released]
  
    
MonsoonBiophysMatNEM <- as.matrix(rbind(dcast(MonsoonNormConn[source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]][monsoon=="NEM" & source != "unknown", .(source, dest, monsoon_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[
    ,-"source"], #remove the source column after casting
      dcast(MonsoonNormConn[monsoon=="NEM"][dest %in% AddDestGenAnnual[year==2014, offs_site]][source == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
MonsoonBiophysMatNEM[is.na(MonsoonBiophysMatNEM)] <- 0 #change NAs to zeros
#dim(MonsoonBiophysMatNEM)

MonsoonBiophysMatSWM <- as.matrix(rbind(dcast(MonsoonNormConn[source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]][monsoon=="SWM" & source != "unknown", .(source, dest, monsoon_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[
    ,-"source"], #remove the source column after casting
      dcast(MonsoonNormConn[monsoon=="SWM"][dest %in% AddDestGenAnnual[year==2014, offs_site]][source == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
MonsoonBiophysMatSWM[is.na(MonsoonBiophysMatSWM)] <- 0 #change NAs to zeros
#dim(MonsoonBiophysMatSWM)

#fit the kernels
    #fit the kernels, get the biophysical data together
biophys_par_data2012 <- list(Distances=Distances, Assignments=AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))

#make a summary table for all time frames of simulation data
SimKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 8))
setDT(SimKernelsInt)
setnames(SimKernelsInt, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[1] <- "annual"
SimKernelsInt$time_id[1] <- "2012"
SimKernelsInt$k[1] <-  BestK2012
SimKernelsInt$theta[1] <- BestTheta2012
SimKernelsInt$mdd[1] <- MDD2012
SimKernelsInt$med[1] <- Med2012
SimKernelsInt$dist90[1] <- Dist90_2012
SimKernelsInt$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[2] <- "annual"
SimKernelsInt$time_id[2] <- "2013"
SimKernelsInt$k[2] <-  BestK2013
SimKernelsInt$theta[2] <- BestTheta2013
SimKernelsInt$mdd[2] <- MDD2013
SimKernelsInt$med[2] <- Med2013
SimKernelsInt$dist90[2] <- Dist90_2013
SimKernelsInt$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[3] <- "annual"
SimKernelsInt$time_id[3] <- "2014"
SimKernelsInt$k[3] <-  BestK2014
SimKernelsInt$theta[3] <- BestTheta2014
SimKernelsInt$mdd[3] <- MDD2014
SimKernelsInt$med[3] <- Med2014
SimKernelsInt$dist90[3] <- Dist90_2014
SimKernelsInt$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[4] <- "interannual"
SimKernelsInt$time_id[4] <- "2012-4"
SimKernelsInt$k[4] <-  BestK2012_4
SimKernelsInt$theta[4] <- BestTheta2012_4
SimKernelsInt$mdd[4] <- MDD2012_4
SimKernelsInt$med[4] <- Med2012_4
SimKernelsInt$dist90[4] <- Dist90_2012_4
SimKernelsInt$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[5] <- "season"
SimKernelsInt$time_id[5] <- "NEM"
SimKernelsInt$k[5] <-  BestKNEM
SimKernelsInt$theta[5] <- BestThetaNEM
SimKernelsInt$mdd[5] <- MDDNEM
SimKernelsInt$med[5] <- MedNEM
SimKernelsInt$dist90[5] <- Dist90_NEM
SimKernelsInt$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[6] <- "season"
SimKernelsInt$time_id[6] <- "SWM"
SimKernelsInt$k[6] <-  BestKSWM
SimKernelsInt$theta[6] <- BestThetaSWM
SimKernelsInt$mdd[6] <- MDDSWM
SimKernelsInt$med[6] <- MedSWM
SimKernelsInt$dist90[6] <- Dist90_SWM
SimKernelsInt$log_like[6] <- logLik(SimSWMFit)[1]

    SimKernelsInt$iteration <- i
    SimKernels <- rbind(SimKernels, SimKernelsInt)
    setTxtProgressBar(pb, i)
    
    
 #also bootstrap the parentage kernels   
    }
close(pb)

fwrite(SimKernels, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalKernels08DayPLD.csv")

In [ ]:
#bootstrap the parentage kernels
#bootstrap the particle data and re-fit kernels
num <- nrow(SimConn)

#make sure all possible routes are represented
AddDestSim <- rbind(rbindlist(list(unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2012],
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2013], 
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2014]))[, monsoon := "NEM"],
    rbindlist(list(unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2012],
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2013], 
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2014]))[, monsoon := "SWM"])

AddDestAllYearsSim <- unique(AddDestSim, by=c("source","dest"))[, -c("year","monsoon")] 

AddDestAnnualSim <- unique(AddDestSim, by=c("source","dest", "year"))[, -"monsoon"] 

AddDestMonsoonSim <- unique(AddDestSim[, -"year"], by=c("source", "dest", "monsoon"))
AddDestAnuualMonsoonSim <- unique(AddDestSim, by=c("source", "dest","year", "monsoon"))

##add in sites that aren't represented in every year by the genetics for the biophysical PARENTAGE matrix used to fit the biophysical kernel
AddDestGenAnnual <- rbindlist(list(unique(cbind(SurveyData[year==2012 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2012 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2012],  #what destinations were sampled, for use with unassigned table
unique(cbind(SurveyData[year==2013 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2013 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2013],
unique(cbind(SurveyData[year==2014 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2014 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2014]))

AddDestGen <- cbind(PropSamp[prop_anem_samp > 0 & year==2014, .(site)][, .(offs_site=site)], PropSamp[prop_anem_samp > 0 & year==2014, .(site)][, .(par_site=site)])

AddDestGenMonsoon <- rbind(cbind(AddDestGen, data.table(monsoon = rep("NEM",nrow(AddDestGen)))), cbind(AddDestGen, data.table(monsoon = rep("SWM",nrow(AddDestGen)))))

#read in all data needed for kernel fitting
#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv") 
Centroids$site <- gsub("_full", "", Centroids$site, fixed=TRUE)
Centroids$site <- gsub("_ten_per_cover", "", Centroids$site, fixed=TRUE)

Centroids <- Centroids %>%
    arrange(site)

#setorder(Centroids, site)#warning! This sets order based on site, and then lat/lon. So the table is not alphabetical by site, but that's fine as long as all of the "sampled_reef" vectors reflect this, so that reef_sizes, distance, and sampled_reefs match up by row/col index 
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- unique(fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")[
    , .(site, year=end_year, prop_anem_samp=total_prop_hab_sampled_anems_tidied)][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , prop_anem_samp := sum(prop_anem_samp), by=c("site", "year")], by=c("site", "year"))[ #collapse magbangons to match ROMS data
    site=="Sitio Lonas" & year %in% c(2012, 2013, 2014), prop_anem_samp :=1][site=="Caridad Proper" & year %in% c(2013, 2014), prop_anem_samp :=1]

### List of source locations
SitesSource <- Centroids

### List of destination locations
SitesDest <- Centroids

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- DistMatm*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
Area$site <- gsub("_ten_per_cover", "", Area$site, fixed=TRUE)

reef_sizes <- as.matrix(Area$kmsq)

#make a site index table, use this for Sampled_reefs input in kernel fitting
SiteIndex <- unique(Centroids %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel 
SiteIndexBioPhys <- unique(Centroids %>% arrange(site), by="site")[site != "CAI"][, index := .I] #add the row number as the unique site index, take CAI out for biophysical likelihood function

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0
#setnames(SurveyData, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#Allison's abundance time series data 
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "~/oceanography/empirical_data/genetics/females_df_F.RData")
load("~/oceanography/empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)
setnames(Abundance, "nF", "num_females")
Abundance <- unique(Abundance[site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
            , num_females := sum(num_females), by=c("site", "year")], by=c("site", "year"))
#join the survey sampling tables together
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0


SurveyData <- Abundance[, c("year", "site", "num_females")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this
#quick check that all components are in the same, alphabetical order
sum(which(SiteIndex$site==Area$site)==FALSE) #needs to be 0!! sites have to be in the same order
sum(which(Area$site==Centroids$site)==FALSE) #needs to be 0!! sites have to be in the same order

#start bootstrapping
SimKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), log_like=numeric(), iteration=numeric())

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

for(i in 1:1000){
    
    SimConnInt <- SimConn[, .SD[sample(.N, num, replace=TRUE)]] #sample the particle data
    
#format the matrices

    AllYearsRec <- SimConnInt[ , .(total_particles_rec = .N), by= c("source","dest")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
    AllYearsRelease <- unique(SimConnInt[, .(total_particles_released = as.numeric(daily_particles_released)*as.numeric(total_release_days)), by= c("source")], by="source") #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs
    
    AllYearsRecInt <- rbind(AddDestAllYearsSim[!AllYearsRec, on =.(source, dest)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
        , total_particles_rec:=0 ], AllYearsRec) 
    
    AllYearsReleaseInt <- rbind(AddDestAllYearsSim[!AllYearsRelease, on =.(source)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
        , total_particles_released:=0 ][, -"dest"], AllYearsRelease) 
    
    #collapse CAI and "Other" together into "unknown" before making the normalized matrices
    AllYearsRec <- unique(AllYearsRecInt[source== "Other" | source == "CAI", source := "unknown"][
        , total_particles_rec := sum(total_particles_rec),  by=c("source", "dest")], by=c("source", "dest"))
    AllYearsRelease <- unique(AllYearsReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
        , total_particles_released := sum(total_particles_released), by="source"], by="source")
    
    #join recruited and released tables together and make a column for the normalized values
    AllYearsNormConn <-  AllYearsRelease[AllYearsRec, on="source"][
        , source_norm_rec := total_particles_rec/total_particles_released]

    #make a parentage matrix for the whole biophysical results- CAI included with Other as "unknown", use this to fit a kernel
FullBiophysMat <- as.matrix(rbind(dcast(AllYearsNormConn[source %in% AddDestGenAnnual[, par_site] & dest %in% AddDestGenAnnual[, offs_site]][source !="unknown", .(source, dest, total_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="total_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
      dcast(AllYearsNormConn[dest %in% AddDestGenAnnual[, offs_site]][source  == "unknown", .(source, dest, total_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="total_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
FullBiophysMat[is.na(FullBiophysMat)] <- 0 #change NAs to zerosdim(FullBiophysMat)
#dim(FullBiophysMat)
#parentage matrix by years and monsoons
    
#make annual matrices with all of the particle data
AnnualRec <- SimConnInt[ , .(annual_particles_rec = .N), by= c("source","dest", "year")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
AnnualRelease <- unique(SimConnInt[, .(annual_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_annual)), by= c("source", "year")], by= c("source", "year")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

AnnualRecInt <- rbind(AddDestAnnualSim[!AnnualRec, on =.(source, dest, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , annual_particles_rec:=0 ], AnnualRec) 

AnnualReleaseInt <- rbind(AddDestAnnualSim[!AnnualRelease, on =.(source, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , annual_particles_released:=0 ][, -"dest"], AnnualRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
AnnualRec <- unique(AnnualRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , annual_particles_rec := sum(annual_particles_rec), by=c("source", "dest", "year")], by=c("source", "dest", "year"))
AnnualRelease <- unique(AnnualReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , annual_particles_released := sum(annual_particles_released), by=c("source", "year")], by=c("source", "year"))

#join recruited and released tables together and make a column for the normalized values
AnnualNormConn <-  AnnualRelease[AnnualRec, on=c("source", "year")][
    , source_norm_rec := annual_particles_rec/annual_particles_released]

AnnualBiophysMat2012 <- as.matrix(rbind(dcast(AnnualNormConn[year==2012][source %in% AddDestGenAnnual[year==2012, par_site] & dest %in% AddDestGenAnnual[year==2012, offs_site]][source !="unknown", .(source, dest, annual_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="annual_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(AnnualNormConn[year==2012][dest %in% AddDestGenAnnual[year==2012, offs_site]][source  == "unknown", .(source, dest, annual_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="annual_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
AnnualBiophysMat2012[is.na(AnnualBiophysMat2012)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2012)

AnnualBiophysMat2013 <- as.matrix(rbind(dcast(AnnualNormConn[year==2013][source %in% AddDestGenAnnual[year==2013, par_site] & dest %in% AddDestGenAnnual[year==2013, offs_site]][source !="unknown", .(source, dest, annual_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="annual_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(AnnualNormConn[year==2013][dest %in% AddDestGenAnnual[year==2013, offs_site]][source  == "unknown", .(source, dest, annual_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="annual_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
AnnualBiophysMat2013[is.na(AnnualBiophysMat2013)] <- 0 #change NAs to zeros
#dim(AnnualBiophysMat2013)

AnnualBiophysMat2014 <- as.matrix(rbind(dcast(AnnualNormConn[year==2014][source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]][source !="unknown", .(source, dest, annual_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="annual_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(AnnualNormConn[year==2014][dest %in% AddDestGenAnnual[year==2014, offs_site]][source  == "unknown", .(source, dest, annual_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="annual_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
AnnualBiophysMat2014[is.na(AnnualBiophysMat2014)] <- 0 #change NAs to zeros
#dim(AnnualBiophysMat2014)

MonsoonRec <- SimConnInt[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConnInt[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon")], by= c("source", "monsoon")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRec <- SimConnInt[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConnInt[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon")], by= c("source", "monsoon")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRecInt <- rbind(AddDestMonsoonSim[!MonsoonRec, on =.(source, dest, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_rec:=0 ], MonsoonRec) 

MonsoonReleaseInt <- rbind(AddDestMonsoonSim[!MonsoonRelease, on =.(source, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_released:=0 ][, -"dest"], MonsoonRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
MonsoonRec <- unique(MonsoonRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , monsoon_particles_rec := sum(monsoon_particles_rec), by=c("source", "dest", "monsoon")], by=c("source", "dest", "monsoon"))
MonsoonRelease <- unique(MonsoonReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , monsoon_particles_released := sum(monsoon_particles_released), by=c("source", "monsoon")], by=c("source", "monsoon"))

#join recruited and released tables together and make a column for the normalized values
MonsoonNormConn <-  MonsoonRelease[MonsoonRec, on=c("source", "monsoon")][
    , source_norm_rec := monsoon_particles_rec/monsoon_particles_released]
  
    
MonsoonBiophysMatNEM <- as.matrix(rbind(dcast(MonsoonNormConn[source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]][monsoon=="NEM" & source != "unknown", .(source, dest, monsoon_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[
    ,-"source"], #remove the source column after casting
      dcast(MonsoonNormConn[monsoon=="NEM"][dest %in% AddDestGenAnnual[year==2014, offs_site]][source == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
MonsoonBiophysMatNEM[is.na(MonsoonBiophysMatNEM)] <- 0 #change NAs to zeros
#dim(MonsoonBiophysMatNEM)

MonsoonBiophysMatSWM <- as.matrix(rbind(dcast(MonsoonNormConn[source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]][monsoon=="SWM" & source != "unknown", .(source, dest, monsoon_particles_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[
    ,-"source"], #remove the source column after casting
      dcast(MonsoonNormConn[monsoon=="SWM"][dest %in% AddDestGenAnnual[year==2014, offs_site]][source == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
MonsoonBiophysMatSWM[is.na(MonsoonBiophysMatSWM)] <- 0 #change NAs to zeros
#dim(MonsoonBiophysMatSWM)

#fit the kernels
    #fit the kernels, get the biophysical data together
biophys_par_data2012 <- list(Distances=Distances, Assignments=AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))

#make a summary table for all time frames of simulation data
SimKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 8))
setDT(SimKernelsInt)
setnames(SimKernelsInt, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[1] <- "annual"
SimKernelsInt$time_id[1] <- "2012"
SimKernelsInt$k[1] <-  BestK2012
SimKernelsInt$theta[1] <- BestTheta2012
SimKernelsInt$mdd[1] <- MDD2012
SimKernelsInt$med[1] <- Med2012
SimKernelsInt$dist90[1] <- Dist90_2012
SimKernelsInt$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[2] <- "annual"
SimKernelsInt$time_id[2] <- "2013"
SimKernelsInt$k[2] <-  BestK2013
SimKernelsInt$theta[2] <- BestTheta2013
SimKernelsInt$mdd[2] <- MDD2013
SimKernelsInt$med[2] <- Med2013
SimKernelsInt$dist90[2] <- Dist90_2013
SimKernelsInt$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[3] <- "annual"
SimKernelsInt$time_id[3] <- "2014"
SimKernelsInt$k[3] <-  BestK2014
SimKernelsInt$theta[3] <- BestTheta2014
SimKernelsInt$mdd[3] <- MDD2014
SimKernelsInt$med[3] <- Med2014
SimKernelsInt$dist90[3] <- Dist90_2014
SimKernelsInt$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[4] <- "interannual"
SimKernelsInt$time_id[4] <- "2012-4"
SimKernelsInt$k[4] <-  BestK2012_4
SimKernelsInt$theta[4] <- BestTheta2012_4
SimKernelsInt$mdd[4] <- MDD2012_4
SimKernelsInt$med[4] <- Med2012_4
SimKernelsInt$dist90[4] <- Dist90_2012_4
SimKernelsInt$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[5] <- "season"
SimKernelsInt$time_id[5] <- "NEM"
SimKernelsInt$k[5] <-  BestKNEM
SimKernelsInt$theta[5] <- BestThetaNEM
SimKernelsInt$mdd[5] <- MDDNEM
SimKernelsInt$med[5] <- MedNEM
SimKernelsInt$dist90[5] <- Dist90_NEM
SimKernelsInt$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernelsInt$time_scale[6] <- "season"
SimKernelsInt$time_id[6] <- "SWM"
SimKernelsInt$k[6] <-  BestKSWM
SimKernelsInt$theta[6] <- BestThetaSWM
SimKernelsInt$mdd[6] <- MDDSWM
SimKernelsInt$med[6] <- MedSWM
SimKernelsInt$dist90[6] <- Dist90_SWM
SimKernelsInt$log_like[6] <- logLik(SimSWMFit)[1]

    SimKernelsInt$iteration <- i
    SimKernels <- rbind(SimKernels, SimKernelsInt)
    setTxtProgressBar(pb, i)
    
    
 #also bootstrap the parentage kernels   
    }
close(pb)

#fwrite(SimKernels, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalKernels08DayPLD.csv")

In [ ]:
num <- 37
TotalPar2012_4 <- fread(file="~/oceanography/empirical_data/genetics/parentage_table_2012-14.csv")
TotalPar2012_4 <- unique(TotalPar2012_4[offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
            , n_matches := sum(n_matches), by=c("offs_site", "par_site", "year")], by=c("offs_site", "par_site", "year"))
test <- TotalPar2012_4
TotalPar2012_4 <- TotalPar2012_4[rep(seq_len(nrow(TotalPar2012_4)), TotalPar2012_4$n_matches), 1:ncol(TotalPar2012_4)][, -"n_matches"]
TotalPar2012_4 <- TotalPar2012_4[, .SD[sample(.N, num, replace=TRUE)]][, .(n_matches= .N), by=c("offs_site", "par_site", "year")]


In [ ]:
#bootstrap event matching
#start bootstrapping
EventMatch <- data.table(time_scale=character(), LL=numeric(), iteration=numeric())

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

TotalPar2012_4 <- fread(file="~/oceanography/empirical_data/genetics/parentage_table_2012-14.csv")
TotalPar2012_4 <- unique(TotalPar2012_4[offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
            , n_matches := sum(n_matches), by=c("offs_site", "par_site", "year")], by=c("offs_site", "par_site", "year"))
TotalPar2012_4 <- TotalPar2012_4[rep(seq_len(nrow(TotalPar2012_4)), TotalPar2012_4$n_matches), 1:ncol(TotalPar2012_4)][, -"n_matches"]

sampled_reefs_vec <- as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]) #***I think these index values need to match dimensions of parentage matrix- filter out sand flats and other unsampled sites***
pop_size_vec <- as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]) #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
#BioPhysMat <- as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]) #source normalized biophysical connectivity matrix. In Eqn. S3.4, this is m ajt/r a (*should it be r at? As in all particles released in time period t?)
prop_samp_vec <- as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)])#vector of proportion of habitat sampled for all reefs in time period t- I think that I should be coming up with a different value for this, but for now this will help me code the function

#for(i in 1:1000){


num <- 37
TotalPar2012_4 <- fread(file="~/oceanography/empirical_data/genetics/parentage_table_2012-14.csv")
TotalPar2012_4 <- unique(TotalPar2012_4[offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
            , n_matches := sum(n_matches), by=c("offs_site", "par_site", "year")], by=c("offs_site", "par_site", "year"))
TotalPar2012_4 <- TotalPar2012_4[rep(seq_len(nrow(TotalPar2012_4)), TotalPar2012_4$n_matches), 1:ncol(TotalPar2012_4)][, -"n_matches"]

TotalPar2012_4 <- TotalPar2012_4[, .SD[sample(.N, num, replace=TRUE)]][, .(n_matches= .N), by=c("offs_site", "par_site", "year")]

#add in all the sampled sites and numbers of recruits sampled at each site
TotalParInt <- unique(TotalPar2012_4[, num_matches := sum(n_matches), by=c("year", "offs_site")][, -"par_site"], by=c("year", "offs_site"))[, -"n_matches"]
#sum(TotalParInt$num_matches) #should be 37

TotalUnassigned2012_4 <- TotalParInt[SurveyData[year %in% c(2012, 2013, 2014)], on=.(offs_site=site, year)]
TotalUnassigned2012_4 [is.na(TotalUnassigned2012_4 )] <- 0

#sum(TotalUnassigned2012_4 $n_offs_gen) #should be 394
#sum(TotalUnassigned2012_4 $num_matches) #should be 37
#nrow(unique(TotalUnassigned2012_4 , by="offs_site"))#should be 18, so that every site is represented so that the years are all 18*18 sites
TotalUnassigned2012_4 <- TotalUnassigned2012_4[, num_unassigned := n_offs_gen-num_matches, by=c("year", "offs_site")][
    , .(offs_site, year, num_unassigned)]
#sum(TotalUnassigned2012_4$num_unassigned) #should be 357

#add in sites that aren't represented in as both parent and offspring possible site, based on assignments
AddDestGen <- rbindlist(list(unique(cbind(SurveyData[year %in% c(2012, 2013, 2014) ][, .(offs_site=site)], 
                  SurveyData[year %in% c(2012, 2013, 2014)][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2012],  #what destinations were sampled, for use with unassigned table
unique(cbind(SurveyData[year %in% c(2012, 2013, 2014) ][, .(offs_site=site)], 
                  SurveyData[year %in% c(2012, 2013, 2014)][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2013],
unique(cbind(SurveyData[year %in% c(2012, 2013, 2014) ][, .(offs_site=site)], 
                  SurveyData[year %in% c(2012, 2013, 2014)][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2014]))

TotalPar2012_4 <- rbind(AddDestGen[!TotalPar2012_4, on =.(par_site, offs_site, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , n_matches:=0 ], TotalPar2012_4[,-"num_matches"])  #add the parentage column, add back into the parentage table but drop the num_matches column that's a summary column I used to make the unassigned table 
#sum(TotalPar2012_4$n_matches)#should be 37 still

#format genetic parentage matrices for each year and all years combined
GenMat2012 <- as.matrix(rbind(dcast(TotalPar2012_4[year==2012, .(par_site, offs_site, n_matches)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(par_site, offs_site)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , par_site ~ offs_site, value.var="n_matches")[
    ,-"par_site"], #remove the source column after casting
      t(as.matrix(TotalUnassigned2012_4[year==2012][order(offs_site)][, .(num_unassigned)])), use.names=FALSE))
GenMat2012[is.na(GenMat2012)] <- 0

GenMat2013 <- as.matrix(rbind(dcast(TotalPar2012_4[year==2013, .(par_site, offs_site, n_matches)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(par_site, offs_site)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , par_site ~ offs_site, value.var="n_matches")[
    ,-"par_site"], #remove the source column after casting
      t(as.matrix(TotalUnassigned2012_4[year==2013][order(offs_site)][, .(num_unassigned)])), use.names=FALSE))
GenMat2013[is.na(GenMat2013)] <- 0

GenMat2014 <- as.matrix(rbind(dcast(TotalPar2012_4[year==2014, .(par_site, offs_site, n_matches)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(par_site, offs_site)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , par_site ~ offs_site, value.var="n_matches")[
    ,-"par_site"], #remove the source column after casting
      t(as.matrix(TotalUnassigned2012_4[year==2014][order(offs_site)][, .(num_unassigned)])), use.names=FALSE))
GenMat2014[is.na(GenMat2014)] <- 0

GenMat2012_4 <- as.matrix(rbind(dcast(TotalPar2012_4[, .(par_site, offs_site, n_matches)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(par_site, offs_site)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , par_site ~ offs_site, value.var="n_matches", fun.aggregate = sum)[
    ,-"par_site"], #remove the source column after casting
      t(as.matrix(TotalUnassigned2012_4[, .(num_unassigned=sum(num_unassigned)), by="offs_site"][order(offs_site)][, .(num_unassigned)])), use.names=FALSE))
GenMat2012_4[is.na(GenMat2012_4)] <- 0

#testing
#for monsoon seasons
MonsoonRecSampPar <- unique(fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")[
    , .(offs_site=site, par_site, monsoon=season)][offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
    , num_gen := (.N), by=c("offs_site", "monsoon")][
    !is.na(par_site), n_matches := as.numeric((.N)), by=c("offs_site", "par_site", "monsoon")], by=c("offs_site", "par_site", "monsoon"))

#sum(unique(MonsoonRecSamp, by=c("offs_site","monsoon"))[, num_gen]) #should be 256MonsoonRecSamp <- unique(MonsoonRecSamp, by=c("site", "par_site", "monsoon", "year"))[
MonsoonRecSampPar <- unique(MonsoonRecSampPar, by=c("offs_site", "par_site", "monsoon"))[
    is.na(par_site), n_matches := 0, by=c("offs_site", "par_site", "monsoon")][
    , .(offs_site, par_site, monsoon, num_gen, n_matches)]
#sum(unique(MonsoonRecSampPar, by=c("offs_site","monsoon"))[, num_gen]) #should be 256
#sum(unique(MonsoonRecSampPar, by=c("offs_site", "par_site","monsoon"))[, n_matches]) #should be 17

#total recruits genotyped in each year and season at each site
MonsoonRecSamp <- unique(MonsoonRecSampPar, by=c("offs_site","monsoon"))[,-c("par_site", "n_matches")] #should be 256

#total parentage matches along each route in each year and season
ParMonsoon <- MonsoonRecSampPar[n_matches >0, -"num_gen"]
#print a summary table to check 
#ParMonsoon[, .(n_matches=sum(n_matches)) , by=c("monsoon")]
#sum(AnnualParMonsoon$n_matches) #should be 17

MonsoonParInt <- ParMonsoon[, .(num_matches = sum(n_matches)), by=c("monsoon", "offs_site")]#sum(MonsoonParInt$num_matches) #should be 17

#join all genotyped and assigned to get unassigned

MonsoonUnassigned <- MonsoonParInt[MonsoonRecSamp, on=.(offs_site, monsoon)]
MonsoonUnassigned[is.na(MonsoonUnassigned)] <- 0
MonsoonUnassigned[, num_unassigned := num_gen-num_matches]
#sum(MonsoonUnassigned$num_gen) #should be 256
#sum(MonsoonUnassigned$num_matches) #should be 17
MonsoonUnassigned <- MonsoonUnassigned[, .(offs_site,monsoon, num_unassigned)]

#AnnualParMonsoon #for assigned in matrix
#MonsoonUnassigned #for unassigned in matrix

#add in sites that aren't represented in every year
AddDestGen <- unique(cbind(SurveyData[year %in% c(2012, 2013, 2014) ][, .(offs_site=site)], 
                  SurveyData[year %in% c(2012, 2013, 2014)][ , .(par_site=site)]), by=c("offs_site", "par_site"))

AddDestGenMonsoon <- rbind(cbind(AddDestGen, data.table(monsoon = rep("NEM",nrow(AddDestGen)))), cbind(AddDestGen, data.table(monsoon = rep("SWM",nrow(AddDestGen)))))

ParMonsoon <- rbind(AddDestGenMonsoon[!ParMonsoon, on =.(par_site, offs_site, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , n_matches:=0 ], ParMonsoon)  #add the parentage column, add back into the parentage table but drop the num_matches column that's a summary column I used to make the unassigned table 
#ParMonsoon <- unique(ParMonsoon[, n_matches := sum(n_matches), by=c("par_site", "offs_site", "monsoon")], by=c("par_site", "offs_site", "monsoon"))
#sum(ParMonsoon$n_matches)#should be 17 still

MonsoonUnassigned <- rbind(AddDestGenMonsoon[!MonsoonUnassigned, on =.( offs_site, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , num_unassigned:=0 ][, -"par_site"], MonsoonUnassigned)  #add the parentage column, add back into the parentage table but drop the num_matches column that's a summary column I used to make the unassigned table 
#MonsoonUnassigned <- unique(MonsoonUnassigned[, num_unassigned := sum(num_unassigned), by=c("offs_site", "monsoon")], by=c("offs_site", "monsoon"))

#sum(MonsoonUnassigned$num_unassigned)#should be 256-17=239

#make into parentage matrices
GenMatNEM <- rbind(dcast(ParMonsoon[monsoon=="NEM"][, .(par_site, offs_site, n_matches)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(par_site, offs_site)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , par_site ~ offs_site, value.var="n_matches")[
    ,-"par_site"], #remove the source column after casting
      t(as.matrix(MonsoonUnassigned[monsoon=="NEM"][order(offs_site)][, .(num_unassigned)])), use.names=FALSE)
GenMatNEM[is.na(GenMatNEM)] <- 0
#dim(KernelGenMatNEM)
#sum(KernelGenMatNEM)
GenMatSWM <- rbind(dcast(ParMonsoon[monsoon=="SWM"][, .(par_site, offs_site, n_matches)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(par_site, offs_site)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , par_site ~ offs_site, value.var="n_matches")[
    ,-"par_site"], #remove the source column after casting
      t(as.matrix(MonsoonUnassigned[monsoon=="SWM"][order(offs_site)][, .(num_unassigned)])), use.names=FALSE)
GenMatSWM[is.na(GenMatSWM)] <- 0

    
#biophysical matrices 
    
#make a normalized version that is num assigned from a source to a destination/ num released from that source
total_release_days <- 687

SimConnInt <- SimConn[, .SD[sample(.N, num, replace=TRUE)]] #sample the particle data

AllYearsRec <- SimConnInt[ , .(total_particles_rec = .N), by= c("source","dest")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
AllYearsRelease <- unique(SimConnInt[, .(total_particles_released = as.numeric(daily_particles_released)*as.numeric(total_release_days)), by= c("source")], by="source") #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

AllYearsRecInt <- rbind(AddDestAllYearsSim[!AllYearsRec, on =.(source, dest)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , total_particles_rec:=0 ], AllYearsRec) 

AllYearsReleaseInt <- rbind(AddDestAllYearsSim[!AllYearsRelease, on =.(source)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , total_particles_released:=0 ][, -"dest"], AllYearsRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
AllYearsRec <- unique(AllYearsRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , total_particles_rec := sum(total_particles_rec),  by=c("source", "dest")], by=c("source", "dest"))
AllYearsRelease <- unique(AllYearsReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , total_particles_released := sum(total_particles_released), by="source"], by="source")

#join recruited and released tables together and make a column for the normalized values
AllYearsNormConn <-  AllYearsRelease[AllYearsRec, on="source"][
    , source_norm_rec := total_particles_rec/total_particles_released]

#check that they sum to =< 1
#AllYearsNormConn[,sum(source_norm_rec), by="source"]#nothing should be greater than 1. It isn't- great

#make sure all possible routes are represented!!*
#cast into wide format
FullBiophysMatNorm <- as.matrix(rbind(dcast(AllYearsNormConn[source !="unknown", .(source, dest, source_norm_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="source_norm_rec")[
    ,-"source"], #remove the source column after casting
      dcast(AllYearsNormConn[source == "unknown", .(source, dest, source_norm_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="source_norm_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
#dim(FullBiophysMatNorm )
FullBiophysMatNorm[is.na(FullBiophysMatNorm)] <- 0 #change NAs to zeros

#make annual matrices with all of the particle data
AnnualRec <- SimConn[ , .(annual_particles_rec = .N), by= c("source","dest", "year")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
AnnualRelease <- unique(SimConn[, .(annual_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_annual)), by= c("source", "year")], by= c("source", "year")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

AnnualRecInt <- rbind(AddDestAnnualSim[!AnnualRec, on =.(source, dest, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , annual_particles_rec:=0 ], AnnualRec) 

AnnualReleaseInt <- rbind(AddDestAnnualSim[!AnnualRelease, on =.(source, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , annual_particles_released:=0 ][, -"dest"], AnnualRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
AnnualRec <- unique(AnnualRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , annual_particles_rec := sum(annual_particles_rec), by=c("source", "dest", "year")], by=c("source", "dest", "year"))
AnnualRelease <- unique(AnnualReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , annual_particles_released := sum(annual_particles_released), by=c("source", "year")], by=c("source", "year"))

#join recruited and released tables together and make a column for the normalized values
AnnualNormConn <-  AnnualRelease[AnnualRec, on=c("source", "year")][
    , source_norm_rec := annual_particles_rec/annual_particles_released]
#check that they sum to =< 1
summary(AnnualNormConn[, .(sum=sum(source_norm_rec)), by=c("year", "source")][, sum])#nothing should exceed 1, it doesn't- great

#cast into wide format for each year

AnnualBiophysMatNorm2012 <- as.matrix(rbind(dcast(AnnualNormConn[year==2012 & source != "unknown", .(source, dest, source_norm_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="source_norm_rec")[
    ,-"source"], #remove the source column after casting
      dcast(AnnualNormConn[year==2012][source == "unknown", .(source, dest, source_norm_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="source_norm_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
#dim(AnnualBiophysMatNorm2012)
AnnualBiophysMatNorm2012[is.na(AnnualBiophysMatNorm2012)] <- 0 #change NAs to zeros

AnnualBiophysMatNorm2013 <- as.matrix(rbind(dcast(AnnualNormConn[year==2013 & source != "unknown", .(source, dest, source_norm_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="source_norm_rec")[
    ,-"source"], #remove the source column after casting
      dcast(AnnualNormConn[year==2013][source == "unknown", .(source, dest, source_norm_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="source_norm_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
#dim(AnnualBiophysMatNorm2013)
AnnualBiophysMatNorm2013[is.na(AnnualBiophysMatNorm2013)] <- 0 #change NAs to zeros

AnnualBiophysMatNorm2014 <- as.matrix(rbind(dcast(AnnualNormConn[year==2014 & source != "unknown", .(source, dest, source_norm_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="source_norm_rec")[
    ,-"source"], #remove the source column after casting
      dcast(AnnualNormConn[year==2014][source == "unknown", .(source, dest, source_norm_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="source_norm_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
#dim(AnnualBiophysMatNorm2014)
AnnualBiophysMatNorm2014[is.na(AnnualBiophysMatNorm2014)] <- 0 #change NAs to zeros

#make monsoon matrices with all of the particle data
MonsoonRec <- SimConn[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConn[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon")], by= c("source", "monsoon")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRec <- SimConn[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConn[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon")], by= c("source", "monsoon")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRecInt <- rbind(AddDestMonsoonSim[!MonsoonRec, on =.(source, dest, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_rec:=0 ], MonsoonRec) 

MonsoonReleaseInt <- rbind(AddDestMonsoonSim[!MonsoonRelease, on =.(source, monsoon)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_released:=0 ][, -"dest"], MonsoonRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
MonsoonRec <- unique(MonsoonRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , monsoon_particles_rec := sum(monsoon_particles_rec), by=c("source", "dest", "monsoon")], by=c("source", "dest", "monsoon"))
MonsoonRelease <- unique(MonsoonReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , monsoon_particles_released := sum(monsoon_particles_released), by=c("source", "monsoon")], by=c("source", "monsoon"))

#join recruited and released tables together and make a column for the normalized values
MonsoonNormConn <-  MonsoonRelease[MonsoonRec, on=c("source", "monsoon")][
    , source_norm_rec := monsoon_particles_rec/monsoon_particles_released]
#check that they sum to =< 1
#summary(MonsoonNormConn[, .(sum=sum(source_norm_rec)), by=c("monsoon", "source")][, sum])#nothing should exceed 1, it doesn't- great

MonsoonBiophysMatNormNEM <- as.matrix(rbind(dcast(MonsoonNormConn[monsoon=="NEM" & source != "unknown", .(source, dest, source_norm_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="source_norm_rec")[
    ,-"source"], #remove the source column after casting
      dcast(MonsoonNormConn[monsoon=="NEM"][source == "unknown", .(source, dest, source_norm_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="source_norm_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
#dim(MonsoonBiophysMatNormNEM)
MonsoonBiophysMatNormNEM[is.na(MonsoonBiophysMatNormNEM)] <- 0 #change NAs to zeros

MonsoonBiophysMatNormSWM <- as.matrix(rbind(dcast(MonsoonNormConn[monsoon=="SWM" & source != "unknown", .(source, dest, source_norm_rec)][ #for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="source_norm_rec")[
    ,-"source"], #remove the source column after casting
      dcast(MonsoonNormConn[monsoon=="SWM"][source == "unknown", .(source, dest, source_norm_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
          order(source, dest)], source ~ dest, value.var="source_norm_rec")[,-"source"]))#bind these two cast wide form data tables (assigned and unassigned particles) and then turn into a matrix to be used in the likelihood functions
#dim(MonsoonBiophysMatNormSWM)
MonsoonBiophysMatNormSWM[is.na(MonsoonBiophysMatNormSWM)] <- 0 #change NAs to zeros

Data2012_4 <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012_4[nrow(GenMat2012_4),]))

Data2012 <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012[1:nrow(GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012[nrow(GenMat2012),]))
Data2013 <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2013[1:nrow(GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2013[nrow(GenMat2013),]))
Data2014 <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2014[1:nrow(GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2014[nrow(GenMat2014),]))

#neg_LL_biophys(Data2012)+neg_LL_biophys(Data2013)+neg_LL_biophys(Data2014)

 DataNEM <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatNEM[1:nrow(GenMatNEM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatNEM[nrow(GenMatNEM),]))

 DataSWM <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatSWM[1:nrow(GenMatSWM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatSWM[nrow(GenMatSWM),]))

EventMatchingInt <- data.table(time_frame= c("interannual", "annual", "avg_monsoonal"), LL=c(neg_LL_biophys(Data2012_4), neg_LL_biophys(Data2012) + neg_LL_biophys(Data2013) + neg_LL_biophys(Data2014), neg_LL_biophys(DataNEM) + neg_LL_biophys(DataSWM)), iteration=c(i, i, i))

        EventMatching <- rbind(EventMatching, EventMatchingInt)
    setTxtProgressBar(pb, i)
    
    
 #also bootstrap the parentage kernels   
  #  }
close(pb)

#fwrite(SimKernels, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalKernels08DayPLD.csv")

In [ ]:
neg_LL_biophys(Data2012_4)

In [ ]:
neg_LL_biophys(Data2012)

In [ ]:
neg_LL_biophys(DataSWM)

In [ ]:
DataSWM

In [ ]:
#testing annual/seasonal kernels WITH SIMULATION RATHER THAN SAMPLING YEAR!!
#make monsoon matrices with all of the particle data
MonsoonRec <- SimConn[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon", "year")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConn[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon", "year")], by= c("source", "monsoon", "year")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRec <- SimConn[ , .(monsoon_particles_rec = .N), by= c("source","dest", "monsoon", "year")] #all particles recruiting along each route FILTER HERE FOR TIME PERIOD***
MonsoonRelease <- unique(SimConn[, .(monsoon_particles_released = as.numeric(daily_particles_released)*as.numeric(num_release_days_seasonal)), by= c("source", "monsoon","year")], by= c("source", "monsoon","year")) #calculate the number of particles released over the time frame by multiplyig the release days by the number of particles released daily. fread() converts big numbers to integers so specify as numeric to avoid integer overflow NAs

MonsoonRecInt <- rbind(AddDestAnnualMonsoonSim[!MonsoonRec, on =.(source, dest, monsoon, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_rec:=0 ], MonsoonRec) 

MonsoonReleaseInt <- rbind(AddDestAnnualMonsoonSim[!MonsoonRelease, on =.(source, monsoon, year)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , monsoon_particles_released:=0 ][, -"dest"], MonsoonRelease) 

#collapse CAI and "Other" together into "unknown" before making the normalized matrices
MonsoonRec <- unique(MonsoonRecInt[source== "Other" | source == "CAI", source := "unknown"][
    , monsoon_particles_rec := sum(monsoon_particles_rec), by=c("source", "dest", "monsoon", "year")], by=c("source", "dest", "monsoon", "year"))
MonsoonRelease <- unique(MonsoonReleaseInt[source== "Other" | source == "CAI", source := "unknown"][
    , annual_monsoon_particles_released := sum(monsoon_particles_released), by=c("source", "monsoon", "year")], by=c("source", "monsoon", "year"))
#something isn't right with the monsoon release table, but we don't need that for the kernel so ignore for now- wait okay replacing released particiles with 0 for sites not getting any recruitment doesn't matter, those are zeros anyways. 


#something isn't right with the monsoon release table, but we don't need that for the kernel so ignore for now
SWMBiophysMat2011 <- as.matrix(rbind(dcast(MonsoonRec[year==2011 & monsoon=="SWM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2011, par_site] & dest %in% AddDestGenAnnual[year==2011, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2011 & monsoon=="SWM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
SWMBiophysMat2011[is.na(SWMBiophysMat2011)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2011)

SWMBiophysMat2012 <- as.matrix(rbind(dcast(MonsoonRec[year==2012 & monsoon=="SWM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2012, par_site] & dest %in% AddDestGenAnnual[year==2012, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2012 & monsoon=="SWM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
SWMBiophysMat2012[is.na(SWMBiophysMat2012)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2012)

SWMBiophysMat2013 <- as.matrix(rbind(dcast(MonsoonRec[year==2013 & monsoon=="SWM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2013, par_site] & dest %in% AddDestGenAnnual[year==2013, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2013 & monsoon=="SWM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
SWMBiophysMat2013[is.na(SWMBiophysMat2013)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2013)

SWMBiophysMat2014 <- as.matrix(rbind(dcast(MonsoonRec[year==2014 & monsoon=="SWM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2014 & monsoon=="SWM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
SWMBiophysMat2014[is.na(SWMBiophysMat2014)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2014)



NEMBiophysMat2011 <- as.matrix(rbind(dcast(MonsoonRec[year==2011 & monsoon=="NEM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2011, par_site] & dest %in% AddDestGenAnnual[year==2011, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2011 & monsoon=="NEM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
NEMBiophysMat2011[is.na(NEMBiophysMat2011)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2011)

NEMBiophysMat2012 <- as.matrix(rbind(dcast(MonsoonRec[year==2012 & monsoon=="NEM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2012, par_site] & dest %in% AddDestGenAnnual[year==2012, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2012 & monsoon=="NEM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
NEMBiophysMat2012[is.na(NEMBiophysMat2012)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2012)

NEMBiophysMat2013 <- as.matrix(rbind(dcast(MonsoonRec[year==2013 & monsoon=="NEM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2013, par_site] & dest %in% AddDestGenAnnual[year==2013, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2013 & monsoon=="NEM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
NEMBiophysMat2013[is.na(NEMBiophysMat2013)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2013)

NEMBiophysMat2014 <- as.matrix(rbind(dcast(MonsoonRec[year==2014 & monsoon=="NEM" & dest != "CAI"][source !="unknown", .(source, dest, monsoon_particles_rec)][ #pulling this bit out for annual monsoon kernels: [source %in% AddDestGenAnnual[year==2014, par_site] & dest %in% AddDestGenAnnual[year==2014, offs_site]]#for assigned particles (not from "Other") keep the source/dest columns that will be expanded into wide form to become the connectivity matrix. Filtering for time period etc can be done in i here.
    order(source, dest)] #keep sites in alphabetical order so the matrix is correctly formatted!
        , source ~ dest, value.var="monsoon_particles_rec")[#use sum to count the matches for each id variable combo, that populated the cells of the matrix
    ,-"source"], #remove the source column after casting
dcast(MonsoonRec[year==2014 & monsoon=="NEM" & dest != "CAI"][source  == "unknown", .(source, dest, monsoon_particles_rec)][ #this is to cast the "unassigned row for the model parentage, which is anyting from "Other"
        order(source, dest)], source ~ dest, value.var="monsoon_particles_rec")[,-"source"]))
NEMBiophysMat2014[is.na(NEMBiophysMat2014)] <- 0 #change NAs to zerosdim(AnnualBiophysMat2014)
###SWM
#fwrite(SWMBiophysMat2011, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2011ForROMSComp08DayPLD.csv")
#fwrite(SWMBiophysMat2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2012ForROMSComp08DayPLD.csv")
#fwrite(SWMBiophysMat2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2013ForROMSComp08DayPLD.csv")
#fwrite(SWMBiophysMat2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2014ForROMSComp08DayPLD.csv")
#
##NEM
#fwrite(NEMBiophysMat2011, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2011ForROMSComp08DayPLD.csv")
#fwrite(NEMBiophysMat2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2012ForROMSComp08DayPLD.csv")
#fwrite(NEMBiophysMat2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2013ForROMSComp08DayPLD.csv")
#fwrite(NEMBiophysMat2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2014ForROMSComp08DayPLD.csv")


In [ ]:
#plot source normalized connectivity
head(AllYearsNormConn)
head(AnnualNormConn)
head(MonsoonNormConn)

In [ ]:
head(AllYearsNormConn)

In [ ]:
SimConn<- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableWithMeta08DayPLD.csv")[year_sampled != 2011] #& dest %!in% unsampled_sites & source %!in% unsampled_sites][ 
SimConn <- ReleaseDays[SimConn, on=.(year_sampled, sim_monsoon)]#join in the info for number of release days in the time frame

head(SimConn)

In [ ]:
head(AllYearsReleaseInt)
head(AllYearsRecInt)

In [ ]:
SitesNS <- read.csv("~/parentage/text_file/sites_NS.txt", stringsAsFactors = F) %>%
    mutate(site=ifelse(site =="S. Magbangon" , "Magbangon", site)) %>%
    filter(site != "N. Magbangon") 
setDT(SitesNS)

WithIndex <- right_join(SitesNS[, .(source_index=index, source=site)], AllYearsNormConn, by="source")
AllYearsNormConnWithIndex <- right_join(SitesNS[, .(dest_index=index, dest=site)], WithIndex, by="dest")[
    , direction := ifelse(dest_index < source_index, "north", ifelse(dest_index> source_index, "south", "self"))]

In [ ]:
summary(AllYearsNormConnWithIndex[direction=="north", source_norm_rec])
summary(AllYearsNormConnWithIndex[direction=="south", source_norm_rec])
summary(AllYearsNormConnWithIndex[direction=="self", source_norm_rec])

In [ ]:
test_mod <- glm(data=AllYearsNormConnWithIndex, total_particles_rec ~ direction, family="poisson")
summary(test_mod)
test_mod <- glm(data=AllYearsNormConnWithIndex, source_norm_rec ~ direction, family="binomial")
summary(test_mod)

In [ ]:
options(scipen = 999)
summary(AnnualNormConn[year==2012, source_norm_rec])
summary(AnnualNormConn[year==2013, source_norm_rec])
summary(AnnualNormConn[year==2014, source_norm_rec])

summary(AnnualNormConn[source==dest & year==2012, source_norm_rec])
summary(AnnualNormConn[source==dest &year==2013, source_norm_rec])
summary(AnnualNormConn[source==dest &year==2014, source_norm_rec])

summary(MonsoonNormConn[source==dest & monsoon=="NEM", source_norm_rec])
summary(MonsoonNormConn[source==dest &monsoon=="SWM", source_norm_rec])

In [ ]:
#AllYearsNormConn$source <-factor(AllYearsNormConn$source, levels=c("Palanas", "Wangag",  "Magbangon", "Cabatoan", "Caridad Cemetery", "Caridad Proper", "Hicgop South",  "Sitio Tugas", "Elementary School",  "Sitio Lonas", "San Agustin", "Poroc San Flower", "Poroc Rose", "Visca",  "Gabas",  "Tamakin Dacot", "Haina", "Sitio Baybayon", "CAI", "Other"))
#AllYearsNormConn$dest <-factor(AllYearsNormConn$dest, levels=c("Palanas", "Wangag", "Magbangon", "Cabatoan", "Caridad Cemetery", "Caridad Proper", "Hicgop South",  "Sitio Tugas", "Elementary School", "Sitio Lonas", "San Agustin", "Poroc San Flower", "Poroc Rose", "Visca", "Gabas", "Tamakin Dacot", "Haina", "Sitio Baybayon", "CAI", "Other"))

AllYearNorm <- ggplot(AllYearsNormConn[source != "Other" & dest != "Other" & source != "CAI" & dest != "CAI" ], aes(source, dest)) + 
    geom_tile(aes(fill = source_norm_rec), color = "grey40")+
    coord_fixed(ratio = 1) +
    theme(axis.text.x=element_text(angle=45,hjust=1))+
    scale_fill_viridis(option="plasma", name="Normalized \nproportion of particles \nrecruiting", na.value="black") +
    geom_abline(slope=1,color = "grey40")#+
  #scale_fill_continuous(na.value = 'salmon')
AllYearNorm
#ggsave(filename="SimmConnMatAllYearsNormSurveyedSites.png", plot= AllYearNorm, path="~/oceanography/script_output/plots/" )


In [ ]:
AllYearsNormConn[source=="Wangag" & dest=="Gabas"]